In [ ]:
#| default_exp datasources.aeronautica
%load_ext autoreload
%autoreload 2

import sys
from pathlib import Path


In [ ]:

# Insert in Path Project Directory
sys.path.insert(0, str(Path().cwd().parent))

# Atualização

> Este módulo atualiza as bases aeronáuticas fazendo novas requisições às APIS

In [ ]:
#| export
import os
from typing import List, Union
from functools import cached_property


import numpy as np
import pandas as pd
from dotenv import find_dotenv, load_dotenv
from fastcore.xtras import Path
from fastcore.parallel import parallel

from extracao.datasources.icao import get_icao
from extracao.datasources.aisweb import get_aisw
from extracao.datasources.aisgeo import get_aisg
from extracao.datasources.redemet import get_redemet
from extracao.datasources.base import Base
from extracao.format import merge_on_frequency

In [ ]:
#| export
load_dotenv(find_dotenv())


True

In [ ]:
__file__ = Path.cwd().parent / "extracao" / "datasources" / "datasources.py"

In [ ]:
#|export
class Aero(Base):
	"""Classe auxiliar para agregar os dados das APIs aeronáuticas"""

	@property
	def stem(self):
		return 'aero'

	@property
	def columns(self):
		return ['Frequência', 'Latitude', 'Longitude', 'Entidade', 'Fonte']

	def extraction(self) -> pd.DataFrame:
		func = lambda f: f()
		radares = pd.read_csv(Path(__file__).parent / 'arquivos' / 'radares.csv')
		radares['Fonte'] = 'RADAR'
		sources = [get_icao, get_aisw, get_aisg, get_redemet]
		dfs = parallel(func, sources, threadpool=True, progress=True)
		dfs.append(radares)
		return dfs

	def _format(
		self,
		dfs: List,  # List with the individual API sources
	) -> pd.DataFrame:  # Processed DataFrame
		if dfs:
			icao = dfs.pop(0)
			for df in dfs:
				icao = merge_on_frequency(icao, df)

			icao = icao.sort_values(by=icao.columns.to_list(), ignore_index=True)
			icao = icao.drop_duplicates(
				subset=['Frequência', 'Latitude', 'Longitude'],
				keep='last',
				ignore_index=True,
			)
			icao = icao.astype(
				{
					'Frequência': 'float',
					'Latitude': 'float',
					'Longitude': 'float',
					'Entidade': 'string',
				}
			)
			icao.loc[np.isclose(icao.Longitude, -472.033447), 'Longitude'] = -47.2033447
			icao.loc[np.isclose(icao.Longitude, 69.934998), 'Longitude'] = -69.934998
			return icao